# Epsim execution / plotting

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle
import itertools
from pathlib import Path
from gengraph import EpsimGraph
from epsim import Epsim
from read_building_csv import read_building_csv

In [ ]:
def get_cumulative(sim_result, prop, rnds):
    if isinstance(sim_result[0][prop], tuple):
        return tuple(map(sum, zip(*[sim_result[rnd][prop] for rnd in rnds])))
    else:
        return sum([sim_result[rnd][prop] for rnd in rnds])


def write_csv(file_path, sim_result):
    with open(file_path, 'w') as f:
        f.write("round")
        for i in range(len(sim_result[0]['states'])):
            f.write(f",state_{i}")
        for key in sim_result[0].keys():
            if key != 'states':
                f.write(f",{key}")
        f.write("\n")

        for rnd, info in enumerate(sim_result):
            f.write(f"{rnd}")
            for num in info['states']:
                f.write(f",{num}")
            for key, val in info.items():
                if key != 'states':
                    f.write(f",{val}")
            f.write("\n")


def mean_states_per_round_over_runs(runs):
    rnd_data_per_run = list(zip(*runs))  # convert dict to list: remove keys
    mean_states_per_rnd_data = [np.mean(list(zip(*[list(run.values())[0] for run in rnd_data])), axis=1) \
                                for rnd_data in rnd_data_per_run]
    return mean_states_per_rnd_data


def mean_cumulative_over_runs(runs):
    run_keys = list(runs[0][0].keys())
    run_keys.remove('states')

    cumulative_over_runs = [[get_cumulative(run, prop, range(len(run))) for prop in run_keys] for run in runs]
    cumulative_mean = list(np.mean(list(zip(*cumulative_over_runs)), axis=1))

    # readd keys
    mean_dict = {prop: cumulative_mean[i] for i, prop in enumerate(run_keys)}

    return mean_dict

## Simple run
as an example of how to use the API

In [ ]:
# Simple run

starttime = time.time()
epsim_graph = EpsimGraph(k=64000, sigma_office=0.5, perc_split_classes=0.0, print_progress=True)
sim = Epsim(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard,
            epsim_graph.school_nbrs_split, epsim_graph.office_nbrs)
read_building_csv(sim, "input_data/salzburg_buildings.csv")
sim_result = sim.run_sim(sim_iters=200, num_start_agents=100,
                         perc_immune_agents={'children': 0.21, 'parents': 0.36}, start_weekday=0,
                         p_spread_family_dict={0: 0.9435}, p_spread_school_dict={0: 0.51}, p_spread_office_dict={0: 0.544},
                         p_detect_child_dict={0: 0.2}, p_detect_parent_dict={0: 0.5},
                         testing_dict={0: {'pcr': {'p': 0.95, 'weekdays': [2]}}, 'antigen': {'p': 0.5, 'weekdays': [0, 4]}},
                         omicron=True, split_stay_home=False, loc_infec_rate=0.12, 
                         avg_visit_times={'supermarket': 20, 'shop': 60, 'restaurant': 90, 'leisure': 120, 'nightlife': 240},
                         need_minutes={'supermarket': 60, 'shop': 120, 'restaurant': 180, 'leisure': 600, 'nightlife': 480},
                         contact_mult={'supermarket': 1, 'shop': 1, 'restaurant': 1, 'leisure': 1, 'nightlife': 1},
                         print_progress=True)
print(f"runtime: {time.time() - starttime}s")

In [ ]:
# print sim statistics
print(f"total infected in supermarket: {get_cumulative(sim_result, 'infected_in_supermarket', range(len(sim_result)))}")
print(f"total infected in shop: {get_cumulative(sim_result, 'infected_in_shop', range(len(sim_result)))}")
print(f"total infected in restaurant: {get_cumulative(sim_result, 'infected_in_restaurant', range(len(sim_result)))}")
print(f"total infected in leisure: {get_cumulative(sim_result, 'infected_in_leisure', range(len(sim_result)))}")
print(f"total infected in nightlife: {get_cumulative(sim_result, 'infected_in_nightlife', range(len(sim_result)))}")

print(f"total infected: {get_cumulative(sim_result, 'infected', range(len(sim_result)))}")
print(f"total infected by children: {get_cumulative(sim_result, 'infected_by_children', range(len(sim_result)))}")
print(f"total infected children: {get_cumulative(sim_result, 'infected_children', range(len(sim_result)))}")
if len(sim_result) > 50:
    print(f"infected first 50 rounds: {get_cumulative(sim_result, 'infected', range(50))}")
    print(f"infected by children first 50 rounds: {get_cumulative(sim_result, 'infected_by_children', range(50))}")
    print(f"infected children first 50 rounds: {get_cumulative(sim_result, 'infected_children', range(50))}")

## Multiple runs per parameter combination
for plotting an averaged diagram of multiple runs to avoid outliners

In [ ]:
# Define simulation parameters
sim_iters = 100

def gen_graph_run_sim(k, sigma_office, num_start_agents, perc_immune_agents, start_weekday, perc_split_classes,
                      p_spread_family_dict, p_spread_school_dict, p_spread_office_dict, p_detect_child_dict, p_detect_parent_dict,
                      testing_dict, omicron, split_stay_home, location_sim, loc_infec_rate, avg_visit_times, need_minutes, contact_mult):
    epsim_graph = EpsimGraph(k, sigma_office, perc_split_classes)
    sim = Epsim(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard, epsim_graph.school_nbrs_split,
                epsim_graph.office_nbrs)
    if location_sim:
        print("location sim enabled")
        read_building_csv(sim, "input_data/salzburg_buildings.csv")
    else:
        print("location sim disabled")
    return sim.run_sim(sim_iters, num_start_agents, perc_immune_agents, start_weekday, p_spread_family_dict,
                       p_spread_school_dict, p_spread_office_dict, p_detect_child_dict, p_detect_parent_dict, testing_dict, 
                       omicron, split_stay_home, loc_infec_rate, avg_visit_times, need_minutes, contact_mult)


sim_params = {
    'k': [64000],
    'sigma_office': [0.5],
    'num_start_agents': [100],
    'perc_immune_agents': [tuple({'children': 0.21, 'parents': 0.36}.items())],
    'start_weekday': [0],
    'perc_split_classes': [0.0],
    'p_spread_family_dict': [tuple({0: 0.9435}.items())],
    'p_spread_school_dict': [tuple({0: 0.51}.items())],
    'p_spread_office_dict': [tuple({0: 0.544}.items())],
    'p_detect_child_dict': [tuple({0: 0.2}.items())],
    'p_detect_parent_dict': [tuple({0: 0.5}.items())],
    'testing_dict': [tuple({0: tuple({'pcr': tuple({'p': 0.95, 'weekdays': (2,)}.items()), \
                    'antigen': tuple({'p': 0.5, 'weekdays': (0, 4)}.items())}.items())}.items())],
    'omicron': [True],
    'split_stay_home': [False],
    'location_sim': [True],
    'loc_infec_rate': [0.07],
    'avg_visit_times': [tuple({'supermarket': 20, 'shop': 60, 'restaurant': 90, 'leisure': 120, 'nightlife': 240}.items())],
    'need_minutes': [tuple({'supermarket': 60, 'shop': 120, 'restaurant': 180, 'leisure': 600, 'nightlife': 480}.items())],
    'contact_mult': [tuple({'supermarket': 1, 'shop': 1, 'restaurant': 1, 'leisure': 1, 'nightlife': 1}.items())]
}

param_combis = list(itertools.product(*sim_params.values()))

In [ ]:
# Execute simulations: multiple runs per parameter combination

num_runs = 2
runs_per_param_combi = {pc: [gen_graph_run_sim(*pc) for i in range(num_runs)] for pc in param_combis}
pickle.dump(runs_per_param_combi, open("runs_per_param_combi.p", 'wb'))

In [ ]:
# Plot mean infections per round over all runs of one parameter combination to avoid outliners

plt.rcParams.update({'font.size': 15})

for pc, runs in runs_per_param_combi.items():
    params = dict(zip(sim_params, pc))
    print(params)
    
    mean_dict = mean_cumulative_over_runs(runs)
    mean_infecs_per_rnd = [sum(states[1:]) for states in mean_states_per_round_over_runs(runs)]
    
    fig, axs = plt.subplots(figsize=(20,13))
    fig.patch.set_facecolor('xkcd:white')
    axs.set_ylim(top=152734)
    axs.set_xlim(0, 100)
    axs.plot(range(len(mean_infecs_per_rnd)), mean_infecs_per_rnd)
    plt.annotate(152734, xy=(-0.065, 152734), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    plt.annotate(int(max(mean_infecs_per_rnd)), xy=(1, int(max(mean_infecs_per_rnd))), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    plt.annotate(int(min(mean_infecs_per_rnd)), xy=(0, int(min(mean_infecs_per_rnd))), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    axs.set_xlabel("Rounds")
    axs.set_ylabel(f"Infected agents (mean of {num_runs} runs)")

    axs.set_title(f"Omicron Variant (p_spread_school={params['p_spread_school_dict'][0]}, " \
                  + f"p_spread_office={params['p_spread_office_dict'][0]})")
    text_params = f"k={params['k']},  " \
                 + f"sigma_office={params['sigma_office']},  " \
                 + f"num_start_agents={params['num_start_agents']},  " \
                 + f"perc_immune_agents={params['perc_immune_agents']},  " \
                 + f"start_weekday={params['start_weekday']},\n" \
                 + f"perc_split_classes={params['perc_split_classes']},  " \
                 + f"p_spread_family_dict={params['p_spread_family_dict']},  " \
                 + f"p_spread_school_dict={params['p_spread_school_dict']},  " \
                 + f"p_spread_office_dict={params['p_spread_office_dict']},\n" \
                 + f"p_detect_child_dict={params['p_detect_child_dict']},  " \
                 + f"p_detect_parent_dict={params['p_detect_parent_dict']},  \n" \
                 + f"testing_dict={params['testing_dict']},  \n" \
                 + f"omicron={params['omicron']},  " \
                 + f"split_stay_home={params['split_stay_home']},  " \
                 + f"location_sim={params['location_sim']}, " \
                 + f"loc_infec_rate={params['loc_infec_rate']}, \n" \
                 + f"avg_visit_times={params['avg_visit_times']}, \n" \
                 + f"need_minutes={params['need_minutes']}, \n" \
                 + f"contact_mult={params['contact_mult']}"
    axs.text(0, -0.33, text_params, transform=axs.transAxes, wrap=True, linespacing=1.5)
    print("avg total persons:")
    for prop, val in mean_dict.items():
        print(f"\t{prop}: {val}")
    plt.show()

In [ ]:
# Write csv of mean values
for pc, runs in runs_per_param_combi.items():
    params = dict(zip(sim_params, pc))
    path = Path(f"csv/p_spread_school_{params['p_spread_school_dict'][0]}_p_spread_office_{params['p_spread_office_dict'][0]}.csv")
    write_csv(path, mean_of_runs(runs))

In [ ]:
# Plot histogram of total infections for all runs of one parameter combination
# This shows how much the total infections of different simulations with the same parameters deviatie due to
# the randomness of the simulation

plt.rcParams.update({'font.size': 12})

for pc, runs in runs_per_param_combi.items():
    print(dict(zip(sim_params, pc)))
    total_infec_per_run = [sum(run[-1]['states'][1:]) for run in runs]
    fig, axs = plt.subplots(figsize=(10,6))
    fig.patch.set_facecolor('xkcd:white')
    axs.set_xlabel("Total infected agents")
    axs.set_ylabel("# Runs")
    axs.hist(total_infec_per_run)
    plt.show()
    print()